<a href="https://colab.research.google.com/github/dzivkovi/LLM_RBAC_experiments/blob/main/10_RBAC_using_Multitenant_VectorDB_Weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Tenancy in Weaviate

### Inspired by: https://www.youtube.com/watch?v=KT2RFMTJKGs

Welcome to this demo notebook! Here, we'll walk you through a small example showcasing the `Multi-Tenancy` function in Weaviate.
Multi-tenancy is a key feature in Weaviate, allowing for the efficient and secure management of data across multiple users or tenants.

## 📖 Further Reading:

- Explore the concept in depth in the [multi-tenancy blog post](https://weaviate.io/blog/multi-tenancy-vector-search).
- Dive into the technical details in our [Weaviate developer documentation](https://weaviate.io/developers/weaviate/manage-data/multi-tenancy#enable-multitenancy).

## Getting started
Before we dive in, there are a few preliminary steps:

1. Set Up a Weaviate Cluster:
This notebook requires a working Weaviate cluster. If you don't have one, fret not! You can set up a free sandbox Weaviate cluster by following our [comprehensive guide](https://weaviate.io/developers/academy/zero_to_mvp/hello_weaviate/set_up).

2. Virtual Environment and Dependencies:
To ensure smooth execution and prevent potential conflicts with your global Python environment, we recommend running the code in a virtual environment. Later in this notebook, we'll guide you through setting up this environment and installing the necessary dependencies.

With these points in mind, let's get started:

In [ ]:
!pip install weaviate-client

## Connecting to Your Weaviate Cluster

To interact with our Weaviate cluster, we'll initialize a client object. Once set up, we'll retrieve the current schemas as a way to verify the connection. Since the cluster is newly created, we expect that no schemas will be present.

In [ ]:
import weaviate
from google.colab import userdata

client = weaviate.Client(
  url="https://multi-tenancy-og68lbsz.weaviate.network",  # URL of your Weaviate instance
  additional_headers={
    "X-OPENAI-Api-Key": userdata.get('OPENAI_API_KEY'), # Replace with your OpenAI key
  }
)

client.schema.delete_all() # Delete all data
is_valid = len(client.schema.get()["classes"]) == 0
print(is_valid)

True


In [ ]:
client.schema.get()  # Get the schema to test connection

{'classes': []}

## Setting Up Multi-Tenancy in a Weaviate Class

In Weaviate, multi-tenancy allows for multiple tenants to securely access and manage data within the same schema. Let's proceed to define a new class that utilizes this feature:

### Define a Multi-Tenancy Enabled Class:
We'll start by creating a class named 'MultiTenancyClass' with the multi-tenancy feature activated.

In [ ]:
from weaviate import Tenant

client.schema.create_class({
    'class': 'MultiTenancyClass',
    'multiTenancyConfig': {'enabled': True}
})

In [ ]:
client.schema.get()  # Get the schema to test connection

{'classes': [{'class': 'MultiTenancyClass',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'multiTenancyConfig': {'enabled': True},
   'properties': [],
   'replicationConfig': {'factor': 1},
   'shardingConfig': {'virtualPerPhysical': 0,
    'desiredCount': 0,
    'actualCount': 0,
    'desiredVirtualCount': 0,
    'actualVirtualCount': 0,
    'key': '',
    'strategy': '',
    'function': ''},
   'vectorIndexConfig': {'skip': False,
    'cleanupIntervalSeconds': 300,
    'maxConnections': 64,
    'efConstruction': 128,
    'ef': -1,
    'dynamicEfMin': 100,
    'dynamicEfMax': 500,
    'dynamicEfFactor': 8,
    'vectorCacheMaxObjects': 1000000000000,
    'flatSearchCutoff': 40000,
    'distance': 'cosine',
    'pq': {'enabled': False,
     'bitCompression': False,
     'segments': 0,
     'centroids': 256,
     'trainingLimit': 100000,
     'encoder': {'type': 'km

### Add Multiple Tenants to the Class:
After establishing the class, we'll associate it with two tenants: 'tenantA' and 'tenantB'

In [ ]:
client.schema.add_class_tenants(
  class_name='MultiTenancyClass',  # The class to which the tenants will be added
  tenants=[Tenant(name='tenantA'), Tenant(name='tenantB')]
)

## Fetching Tenants from a Weaviate Class

To view the tenants associated with a specific class, we can retrieve a list of all the tenants linked to it. Let's do this for our previously created class, 'MultiTenancyClass':

In [ ]:
tenants = client.schema.get_class_tenants(
    class_name='MultiTenancyClass'  # The class from which the tenants will be retrieved
)

print(tenants)
is_valid = len(tenants) == 2
print(is_valid)

[Tenant(name='tenantA', activity_status=<TenantActivityStatus.HOT: 'HOT'>), Tenant(name='tenantB', activity_status=<TenantActivityStatus.HOT: 'HOT'>)]
True


## Assigning Data Objects to Specific Tenants

In Weaviate, data objects can be associated with specific tenants in a multi-tenancy enabled class. Here, we will demonstrate how to create data objects and link them to their respective tenants:

In [ ]:
object_a = client.data_object.create(
      class_name='MultiTenancyClass',  # The class to which the object will be added
      data_object={
          'text': 'This belongs to TenantA'
      },
      tenant='tenantA'  # The tenant to which the object will be added
)

# First object
object_b_1 = client.data_object.create(
      class_name='MultiTenancyClass',
      data_object={
          'text': 'This belongs to TenantB'
      },
      tenant='tenantB'
)

# Second object
object_b_2 = client.data_object.create(
      class_name='MultiTenancyClass',
      data_object={
          'text': 'This also belongs to TenantB'
      },
      tenant='tenantB'
)

## Performing Tenant-Specific Queries

By leveraging the multi-tenancy functionality, we can conduct queries that are specific to individual tenants. This enables us to fetch data solely associated with a designated tenant.

In [ ]:
result_a = (
    client.query.get('MultiTenancyClass', ['text'])
    .with_tenant('tenantA')
    .do()
)

result_b = (
    client.query.get('MultiTenancyClass', ['text'])
    .with_tenant('tenantB')
    .do()
)

print(f"TenantA: {result_a['data']['Get']['MultiTenancyClass']}")
print(f"TenantB: {result_b['data']['Get']['MultiTenancyClass']}")

is_valid = len(result_a["data"]["Get"]["MultiTenancyClass"]) == 1 and len(result_b["data"]["Get"]["MultiTenancyClass"]) == 2
print(is_valid)


TenantA: [{'text': 'This belongs to TenantA'}]
TenantB: [{'text': 'This also belongs to TenantB'}, {'text': 'This belongs to TenantB'}]
True


## Removing Tenants from a Weaviate Class

In situations where specific tenants are no longer required, Weaviate allows us to remove them from a class. This action will only affect the specified tenants, leaving other associated tenants unaffected.

In [ ]:
client.schema.remove_class_tenants(
    class_name='MultiTenancyClass',  # The class from which the tenants will be removed
    tenants=['tenantB', 'tenantX']  # The tenants to be removed. tenantX will be ignored.
)

tenants = client.schema.get_class_tenants(
    class_name='MultiTenancyClass'  # The class from which the tenants will be retrieved
)

print(tenants)
is_valid = len(tenants) == 1
print(is_valid)

[Tenant(name='tenantA', activity_status=<TenantActivityStatus.HOT: 'HOT'>)]
True
